# Phase 2 / Level 2 Trace Viewer
Interactive inspection of model competition, beliefs, and EFE-driven choices.

In [1]:
# pyright: reportUndefinedVariable=false
import sys
from pathlib import Path

# Add repo root to Python path regardless of notebook cwd.
repo_root = Path.cwd()
if repo_root.name == 'notebooks':
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

In [2]:
import numpy as np

try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError as exc:
    raise ModuleNotFoundError(
        "matplotlib is required for trace_viewer_phase2_level2.ipynb. "
        "Install with: pip install matplotlib"
    ) from exc

try:
    import ipywidgets as widgets
except ModuleNotFoundError as exc:
    raise ModuleNotFoundError(
        "ipywidgets is required for trace_viewer_phase2_level2.ipynb. "
        "Install with: pip install ipywidgets"
    ) from exc

from IPython.display import display

from agent.pymdp_agent_phase2_level2 import ActiveInferenceStructureLearningAgent
from env_phase2.io import load_world

In [3]:
trace = []
episode = None

for trial_seed in range(1, 40):
    world = load_world(repo_root / 'env_phase2/worlds/phase2_train_world.json', seed=trial_seed)
    agent = ActiveInferenceStructureLearningAgent(
        policy_precision=5.0,
        model_precision=1.0,
        occam_scale=0.1,
    )
    episode = agent.rollout_episode(world=world, max_steps=10, enable_trace=True)
    trace = episode['trace']
    if len(trace) > 1:
        break

if len(trace) <= 1:
    raise RuntimeError(
        'Trace has only one step, so slider interaction is limited. '
        'Try increasing max_steps or changing seed/world.'
    )

print(
    f"steps={len(trace)} success={episode['success']} "
    f"selected_final={episode['selected_family_final']}"
)

steps=7 success=True selected_final=factorized


In [ ]:
def render_step(i: int):
    tr = trace[i]

    fig, axes = plt.subplots(2, 3, figsize=(14, 8))
    fig.suptitle(
        f"Phase 2 Trace | step={tr['t']} action={tr['action']} selected={tr['selected_family']}",
        fontsize=12,
    )

    ax = axes[0, 0]
    post = tr['model_family_posterior']
    ax.bar(['flat', 'factorized'], post, color=['tab:blue', 'tab:orange'])
    ax.set_ylim(0.0, 1.0)
    ax.set_title('Model posterior')

    ax = axes[0, 1]
    ax.bar(['flat', 'factorized'], [tr['fe_inc_flat'], tr['fe_inc_factorized']], label='FE inc', alpha=0.8)
    ax.bar(['flat', 'factorized'], [tr['complexity_flat'], tr['complexity_factorized']], label='complexity', alpha=0.8)
    ax.set_title('FE increment + complexity')
    ax.legend()

    ax = axes[0, 2]
    ax.bar(['flat', 'factorized'], [tr['score_cum_flat'], tr['score_cum_factorized']])
    ax.set_title('Cumulative score (lower is better)')

    ax = axes[1, 0]
    efe = tr['efe_per_action']
    actions = ['do_nothing', 'climb_obj1', 'climb_obj2']
    colors = ['tab:gray', 'tab:gray', 'tab:gray']
    colors[int(tr['chosen_action'])] = 'tab:red'
    ax.bar(actions, efe, color=colors)
    ax.tick_params(axis='x', rotation=25)
    ax.set_title('EFE per action (selected model)')

    ax = axes[1, 1]
    if tr['selected_family'] == 'flat':
        p_reach = tr['beliefs_flat']['p_reach_obj']
        p_climb = tr['beliefs_flat']['p_climb_obj']
        x = np.arange(2)
        ax.bar(x - 0.15, p_reach, width=0.3, label='p_reach_obj')
        ax.bar(x + 0.15, p_climb, width=0.3, label='p_climb_obj')
        ax.set_xticks(x)
        ax.set_xticklabels(['obj1', 'obj2'])
    else:
        qz = tr['beliefs_factorized']['q_object_type']
        x = np.arange(2)
        ax.bar(x - 0.15, qz[:, 0], width=0.3, label='type0')
        ax.bar(x + 0.15, qz[:, 1], width=0.3, label='type1')
        ax.set_xticks(x)
        ax.set_xticklabels(['obj1', 'obj2'])
    ax.set_ylim(0.0, 1.0)
    ax.set_title('Selected-model latent beliefs')
    ax.legend()

    ax = axes[1, 2]
    dbg = tr.get('world_info', {})
    txt = (
        f"obs: can_reach={tr['observation']['can_reach']}, climb_result={tr['observation']['climb_result']}\n"
        f"debug: agent_h={dbg.get('agent_height_internal', '?')}, target_h={dbg.get('target_height_internal', '?')}\n"
        f"debug slots: s1_obj={dbg.get('slot_1_object_internal', '?')}, s2_obj={dbg.get('slot_2_object_internal', '?')}"
    )
    ax.axis('off')
    ax.text(0.02, 0.95, txt, va='top', family='monospace')
    ax.set_title('Agent-view vs debug-view')

    plt.tight_layout()
    plt.show()

step_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(trace) - 1,
    step=1,
    description='step',
    continuous_update=False,
)
out = widgets.interactive_output(render_step, {'i': step_slider})
display(step_slider, out)

IntSlider(value=0, continuous_update=False, description='step', max=6)

Output()